In [1]:
!pip install python-docx

In [2]:
from apiclient import discovery
from httplib2 import Http
from oauth2client import client, file, tools
import argparse

: 

In [3]:
SCOPES = "https://www.googleapis.com/auth/forms.body"
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"

In [4]:
store = file.Storage("token.json")
creds = None

In [5]:
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets("client_secrets.json", SCOPES)

    # avoid conflits with Colab environment
    flags = argparse.Namespace(auth_host_name='localhost',
                               auth_host_port=[8080, 8090],
                               logging_level='ERROR',
                               noauth_local_webserver=True)
    creds = tools.run_flow(flow, store, flags)


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=827704079413-8lemtt51jbmvkm0igqf2p5u5qo2u7ok7.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.body&access_type=offline&response_type=code

Enter verification code: 4/1AQSTgQEJ7MApQgkrlQESPeGhzJy2TS52hBb_4MD1VdqkvgVkgOiWbw4v1Hg
Authentication successful.


In [6]:
form_service = discovery.build(
    "forms",
    "v1",
    http=creds.authorize(Http()),
    discoveryServiceUrl=DISCOVERY_DOC,
    static_discovery=False,
)

以上内容是配置，连接api，与authorization。不需要修改的的

In [7]:
# create a new Google Form
NEW_FORM = {
    "info": {
        "title": "Feedback on Your Report",
    }
}
result = form_service.forms().create(body=NEW_FORM).execute()

In [8]:
from docx import Document

# read Word file and switch to Markdown format for better readability
def read_word_as_markdown(file_path):
    doc = Document(file_path)
    markdown_lines = []

    for para in doc.paragraphs:
        text = para.text.strip()

        # 忽略空行
        if not text:
            continue

        # 处理加粗
        if any(run.bold for run in para.runs):
            text = f"**{text}**"

        # 处理斜体
        if any(run.italic for run in para.runs):
            text = f"*{text}*"

        # 处理列表项
        if text.startswith("•") or text.startswith("- "):
            text = f"- {text.lstrip('•- ')}"

        markdown_lines.append(text)

    # 用换行符拼接 Markdown 文本
    return "\n\n".join(markdown_lines)


word_file_path = "dave form4 report.docx"
report_markdown = read_word_as_markdown(word_file_path)

In [9]:
# use batchUpdate to add `description`，`email question`，and `Report Preview` title
UPDATE_REQUEST = {
    "requests": [
        {
            "updateFormInfo": {
                "info": {
                    "description": """Dear Candidate,

This questionnaire is a job-seeking analysis report generated based on your previous responses, and it is fully customized. We have broken down the entire report into individual sentences for your evaluation. What you are expected to do is to assess each sentence by answering:

1. Do you agree or not?
2. Is it useful or not?

Your feedback is very important to us. Thank you for your participation!
                    """
                },
                "updateMask": "description"
            }
        },
        # add "Please enter your email address here" question as default
        {
            "createItem": {
                "item": {
                    "title": "Please enter your email address here",
                    "questionItem": {
                        "question": {
                            "required": True,  # set as "required"
                            "textQuestion": {
                                "paragraph": False  # short answer input
                            }
                        }
                    }
                },
                "location": {
                    "index": 0  # add it to the top
                }
            }
        },
        # add "Report Preview" title (text)
        {
            "createItem": {
                "item": {
                    "title": "**You can see your Report Preview here first for better understanding:**",  # bold
                    "textItem": {},  # tell Google Forms that this is a text part（not a question or Section）
                },
                "location": {
                    "index": 1  # add it after the email question
                }
            }
        },
        # insert report content (text)
        {
            "createItem": {
                "item": {
                    "title": "",  # empty
                    "description": report_markdown,  # put the content of Word file into description
                    "textItem": {}  # switch it to normal text
                },
                "location": {
                    "index": 2  # insert the whole part after "Report Preview"
                }
            }
        }
    ]
}

# execute
form_service.forms().batchUpdate(formId=result["formId"], body=UPDATE_REQUEST).execute()


{'replies': [{},
  {'createItem': {'itemId': '71b42b29', 'questionId': ['046fd327']}},
  {'createItem': {'itemId': '0c21d695'}},
  {'createItem': {'itemId': '180d6ce0'}}],
 'writeControl': {'requiredRevisionId': '00000004'}}

In [13]:
import re
from docx import Document  # 读取 .docx

# read file
file_path = "chuqi 2.0 report.docx"
document = Document(file_path)

# Define common abbreviations to prevent incorrect splitting
abbreviations = {
    "U.S.": "US_MARKER",
    "U.K.": "UK_MARKER",
    "e.g.": "EG_MARKER",
    "i.e.": "IE_MARKER",
    "St. ": "ST_MARKER ",  # "St. Louis" needs to retain the space
    "vs.": "VS_MARKER",
}

def protect_abbreviations(text):
    """Protect abbreviations using placeholders"""
    for abbr, marker in abbreviations.items():
        text = text.replace(abbr, marker)
    return text

def restore_abbreviations(text):
    """Restore placeholders to their original abbreviations"""
    for abbr, marker in abbreviations.items():
        text = text.replace(marker, abbr)
    return text

# process report content
all_sentences = []
for para in document.paragraphs:
    if para.text.strip():  # skip empty paragraph
        protected_text = protect_abbreviations(para.text.strip())  # replace the abbreviations
        sentences = re.split(r'(?<=[.?!])\s+', protected_text)  # split by usual rule first
        sentences = [restore_abbreviations(sentence) for sentence in sentences]  # replace by correct format again
        all_sentences.extend(sentences)  # add to sentemces list

# print result
print(all_sentences)


['Job Application Evaluation Report', 'Overall Score: 60 / 100\nSummary:\nYou have been actively job searching for more than 4 months, submitted 500-1000 applications, yet received only 1 online assessment and 0 interviews.', 'This suggests a serious bottleneck in your job search process—likely at the resume screening or networking stage.', 'Your understanding of job search strategies is incomplete, particularly regarding ATS, mass vs. targeted applications, and SME (small-to-medium enterprises) job opportunities.', 'Your resume & LinkedIn need refinement, especially in tailoring and showcasing your projects more effectively.', 'To increase your interview rate, you should aim to achieve 80+ points by improving resume strategy, LinkedIn engagement, and networking tactics.', 'Section 1: Current Status Evaluation', '(Score: 7/10)', 'What You’re Doing Well:\nStrong technical background from Washington University in St. Louis.', 'Two U.S. internships—a valuable asset for job applications.',

add "time" to buffer the api limit every time

In [15]:
import time

# go through all sentences to generate report
current_index = 3
for sentence in all_sentences:
    sentence = sentence.strip().replace("\n", " ")  # remove newline sign to avoid bug

    if re.match(r"(?i)section \d+:?", sentence):  # match Section title
        # create Section title
        section_body = {
            "requests": [
                {
                    "createItem": {
                        "item": {
                            "title": sentence,  # Section title
                            "pageBreakItem": {},  # a new Section
                        },
                        "location": {"index": current_index},
                    }
                }
            ]
        }
        form_service.forms().batchUpdate(formId=result["formId"], body=section_body).execute()
        current_index += 1
    elif sentence.endswith(":"):  # normal title（end with ":"）
        # create Form title
        title_body = {
            "requests": [
                {
                    "createItem": {
                        "item": {
                            "title": sentence,  # normal titles
                            "textItem": {},  # only show as text
                        },
                        "location": {"index": current_index},
                    }
                }
            ]
        }
        form_service.forms().batchUpdate(formId=result["formId"], body=title_body).execute()
        current_index += 1
    else:
        # create 2 questions for each sentences (agreement level + usefulness)
        # q 1：agreement level
        question_body1 = {
            "requests": [
                {
                    "createItem": {
                        "item": {
                            "title": sentence,  # question title
                            "questionItem": {
                                "question": {
                                    "required": True,
                                    "scaleQuestion": {
                                        "low": 1,
                                        "high": 5,
                                        "lowLabel": "Strongly Disagree",
                                        "highLabel": "Strongly Agree",
                                    },
                                }
                            },
                        },
                        "location": {"index": current_index},
                    }
                }
            ]
        }
        form_service.forms().batchUpdate(formId=result["formId"], body=question_body1).execute()
        current_index += 1
        time.sleep(1) # **add 1 second buffer to avoid api limit**

        # q 2: usefulness
        question_body2 = {
            "requests": [
                {
                    "createItem": {
                        "item": {
                            "title": "Do you think the statement/score/suggestions you just read is useful to you?",
                            "questionItem": {
                                "question": {
                                    "required": True,
                                    "choiceQuestion": {  # multiple chioce
                                        "type": "RADIO",  # single chioce
                                        "options": [
                                            {"value": "Yes"},
                                            {"value": "No"}
                                        ],
                                    },
                                }
                            },
                        },
                        "location": {"index": current_index},
                    }
                }
            ]
        }
        form_service.forms().batchUpdate(formId=result["formId"], body=question_body2).execute()
        current_index += 1
        time.sleep(1)  # **add 1 second buffer**

# get completed form link
get_result = form_service.forms().get(formId=result["formId"]).execute()
print("Form URL:", get_result["responderUri"])


Form URL: https://docs.google.com/forms/d/e/1FAIpQLScm8e3XKLEjKQvpPD-m4p6HPGdo3TymRGslAdCZ7Pc78WGINg/viewform
